In [ ]:
#default_exp schema

# Schema module
Schema of the database object

In [ ]:
#hide
import pickle, os

os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
os.environ['DAX_ENDPOINT'] = 'longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'
os.environ['LINEKEY'] = '2uAfV4AoYglUGmKTAk2xNOm0aV2Ufgh1BQPvQl9vJd4'
REGION = 'ap-southeast-1'

In [ ]:
#export
import os

DATABASE_TABLE_NAME = os.environ.get('DATABASE_TABLE_NAME')
INVENTORY_BUCKET_NAME = os.environ.get('INVENTORY_BUCKET_NAME')
INPUT_BUCKET_NAME = os.environ.get('INPUT_BUCKET_NAME')
REGION = os.environ.get('REGION') or 'ap-southeast-1'
ACCESS_KEY_ID = os.environ.get('USER') or None
SECRET_ACCESS_KEY = os.environ.get('PW') or None
LINEKEY= os.environ.get('LINEKEY')
  
try:
  DAX_ENDPOINT = os.environ['DAX_ENDPOINT']
except KeyError as e:
  DAX_ENDPOINT = None
  print(f'dax endpoint missing {e}')
  
print(DAX_ENDPOINT)

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
from villaProductDatabase.database import ProductDatabase
from villaProductDatabase.query import Querier

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
from nicHelper.wrappers import add_method, add_class_method, add_static_method
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, JSONAttribute, BooleanAttribute, BinaryAttribute
from pynamodb.indexes import GlobalSecondaryIndex, AllProjection
from pynamodb.models import Model
from pprint import pformat

## Create index helper function

In [ ]:
#export
def createIndex(name, rangeKeyName= None, indexName = None, HashKeyType = UnicodeAttribute, RangeKeyType = UnicodeAttribute):
  class ReturnSecondaryIndex(GlobalSecondaryIndex):
    class Meta:
      index_name = indexName or name
      projection = AllProjection()
      dax_read_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
      dax_write_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
      read_capacity_units = 1
      write_capacity_units = 1
  setattr(ReturnSecondaryIndex, name, HashKeyType(hash_key = True))
  if rangeKeyName:
    setattr(ReturnSecondaryIndex, rangeKeyName, RangeKeyType(range_key = True))
  return ReturnSecondaryIndex()

In [ ]:
#export
class KeySchema(Model):
  iprcode = NumberAttribute(hash_key=True, default = 0)
  cprcode = NumberAttribute(default = 0, range_key = True)
  oprcode = NumberAttribute(default = 0)
  pr_dpcode = UnicodeAttribute(default = 'none')
  pr_barcode = UnicodeAttribute(default = 'none')
  pr_barcode2 = UnicodeAttribute(default = 'none')
  pr_sucode1 = UnicodeAttribute(default = 'none')
  pr_suref3 = UnicodeAttribute(default= 'none')
  pr_sa_method = UnicodeAttribute(default= 'none')
  sellingPrice = NumberAttribute(default = 0)
  lastUpdate = NumberAttribute( default = 0)
  needsUpdate = UnicodeAttribute(default = 'Y')
  data = JSONAttribute()

  # indexes
  needsUpdateIndex = createIndex('needsUpdate','sellingPrice')
  cprcodeIndex = createIndex('cprcode', 'sellingPrice')
  oprcodeIndex = createIndex('oprcode', 'sellingPrice')
  pr_dpcodeIndex = createIndex('pr_dpcode', 'sellingPrice')
  pr_barcodeIndex = createIndex('pr_barcode', 'sellingPrice')
  pr_barcode2Index = createIndex('pr_barcode2', 'sellingPrice')
  pr_suref3Index = createIndex('pr_suref3', 'sellingPrice')
  pr_sa_methodIndex = createIndex('pr_sa_method', 'sellingPrice')
  cprcodeOnlyIndex = createIndex('cprcode', indexName='cprcodeOnly')
  
  def __repr__(self):
    return pformat(vars(self)["attribute_values"])

# Test

In [ ]:
class Tester(Querier, KeySchema):
  class Meta:
    aws_access_key_id = ACCESS_KEY_ID
    aws_secret_access_key = SECRET_ACCESS_KEY
    table_name = DATABASE_TABLE_NAME
    region = REGION
    billing_mode='PAY_PER_REQUEST'
    dax_read_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
    dax_write_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
  pass

In [ ]:
import pandas as pd
from s3bz.s3bz import S3
import boto3, json, yaml, sys, gzip
df = pd.DataFrame([i.data for i in Tester.scan()])
df.head()

,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_dpcode,pr_engname,pr_ggcode,...,preorder_fix_date,preorder_relative_day,meta_title,meta_keywords,meta_description,depth,priority_score,product_attribute_id,product_attribute,related_products
0,225407,225407,225407,Y,MOKU,True,87,21,KONJAC LINGUINI,010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,241101,241101,241101,Y,BUMILGOCHUJANG,True,06,08,BUMILGOCHUJANG,006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,190100,190100,190100,Y,WHITE SHIMEJI,True,05,19,WHITE SHIMEJI,143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,62644,62644,62644,Y,STUTE APRICOT JAM430,True,07,08,STUTE APRICOT JAM 430 G.,002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,192167,192167,192167,Y,**COLGATE TOTAL CHAR,True,10,09,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(45075, 75)

In [ ]:
import requests
ECOMMERCE_COL_LIST = 'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/dev/product/ecommerceColList.yaml'
onlineList = yaml.load(requests.get(ECOMMERCE_COL_LIST).content)

<ipython-input-11-bcf26bf89ba1>:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  onlineList = yaml.load(requests.get(ECOMMERCE_COL_LIST).content)


In [ ]:
condition = df['master_online'] == True
condition2 = df['hema_name_en'] != ''
onlineDf = df[condition & condition2].loc[:,onlineList]
print('shape is:',onlineDf.shape)
# get size
print('size is:',sys.getsizeof(onlineDf.to_json(orient='split'))/1e6, 'Mb')

shape is: (19108, 11)
size is: 3.272721 Mb


In [ ]:
key = 'onlineData'
bucket = INVENTORY_BUCKET_NAME
path = '/tmp/tmp.json'

In [ ]:
## save to s3
from s3bz.s3bz import S3, ExtraArgs


onlineDf.to_json(path ,orient='split',compression='gzip')

S3.saveFile(key=key,path=path,bucket=bucket,
            ExtraArgs = {**ExtraArgs.gzip, **ExtraArgs.publicRead })

# s3 = boto3.resource('s3')
# onlineDict = onlineDf.to_dict(orient='split')
# onlineJson = json.dumps(onlineDict)
# onlineGzip = gzip.compress(onlineJson.encode())

# with open(path, 'wb') as f:
#   f.write(onlineGzip)
  
# s3.meta.client.upload_file(
#   Filename=path,
#   Bucket=bucket,
#   Key=key,
#   ExtraArgs={'ContentType': 'application/json', 'ContentEncoding':'gzip','ACL':'public-read'})

In [ ]:
onlineDf[onlineDf['hema_name_en'] == '']

,cprcode,iprcode,psqty,pr_country_en,online_category_l1_en,online_category_l2_en,online_category_l3_en,hema_brand_en,hema_sizedesc,hema_name_en,hema_name_th


In [ ]:
onlineDf[onlineDf['cprcode'] == 204731]

,cprcode,iprcode,psqty,pr_country_en,online_category_l1_en,online_category_l2_en,online_category_l3_en,hema_brand_en,hema_sizedesc,hema_name_en,hema_name_th
22068,204731,204731,1,France,Delicatessen & Eggs,Salmon & Seafood,Smoked Seafood,Labeyrie,80g,Salmon Roe Jar,Salmon Roe Jar


In [ ]:
dfReal = pd.read_json('https://product-bucket-dev-manual.s3-ap-southeast-1.amazonaws.com/onlineData',orient='split', compression='gzip')
dfReal[dfReal['cprcode']==242687]

,cprcode,iprcode,psqty,pr_country_en,online_category_l1_en,online_category_l2_en,online_category_l3_en,hema_brand_en,hema_sizedesc,hema_name_en,hema_name_th
3335,242687,242687,1,,Health & Beauty,Body & Facial Skincare,Shower & Body Wash,Glow Lab,400ml,Coconut & Sandalwood Body Wash,
